问题三第一问：
要求我们制定未来三个园区的风电、光伏及储能系统的协调配置方案，确保在最大负荷增长50%的情况下，各园区的能源供需平衡，并进行经济性分析。
具体任务包括：
（1）、制定各园区独立运营和联合运营的风光储协调配置方案。

数学建模和求解运算：

1. 独立运营和联合运营的风光储协调配置方案
假设和参数设置：
负荷增长50%，即各园区的最大负荷变为原来的1.5倍。
风电、光伏电源的配置成本分别为3000元/kW、2500元/kW。
储能系统的成本由功率和能量分别计算：功率单价800元/kW，能量单价1800元/kWh。
投资回报期5年。
分时电价如表1所示。

2.步骤：
确定负荷需求：根据附件1的负荷数据，将各园区的负荷增长50%。
风光电源配置：计算满足新增负荷需求所需的风电和光伏装机容量。
储能系统配置：根据风光发电与负荷的不匹配情况，确定储能系统的功率和容量配置。
经济性分析：计算各方案的总投资成本、运行成本及回报期内的总成本。

3.公式和计算：
风电和光伏的配置容量：根据负荷增长情况和当前配置比例计算。
储能系统的容量计算：假设储能系统的容量应满足某一比例的日最大负荷波动量。
总投资成本：风电、光伏及储能系统的总成本。



代码实现
1. 数据预处理
读取附件1和附件2中的数据，计算各园区的典型日负荷数据和风光发电数据。

关键步骤解释
数据预处理：读取并处理负荷和发电数据，确保数据的准确性和一致性。
风光电源配置：根据负荷需求和发电特性，计算所需的风电和光伏装机容量。
储能系统配置：根据负荷和发电的不匹配情况，确定储能系统的功率和容量配置。
经济性分析：综合考虑风光电源和储能系统的投资成本及运行成本，计算总年化成本。
制定合理的约束条件
储能系统的SOC范围设置为10%-90%，充/放电效率为95%，确保储能系统的运行可靠性。
负荷增长50%，确保负荷增长后的能源供需平衡。
投资回报期设定为5年，符合实际工程投资回报期要求。

整合所有代码如下：

In [3]:
import pandas as pd

# 读取负荷数据
load_data = pd.read_excel('附件1：各园区典型日负荷数据.xlsx')
# 读取风光发电数据
generation_data = pd.read_excel('附件2_new.xlsx')

# 确保时间列不会影响计算
load_data.set_index('时间（h）', inplace=True)
generation_data.set_index('时间（h）', inplace=True)

# 打印列名以确认
print("Load data columns:", load_data.columns)
print("Generation data columns:", generation_data.columns)

# 负荷增长50%
load_data_grown = load_data * 1.5

# 计算每个园区需要的风电和光伏装机容量
def calculate_capacity(load_data, generation_data, wind_cost_per_kw, solar_cost_per_kw):
    capacity = {}
    for park in load_data.columns:
        if 'Load' in park:
            max_load = load_data[park].max()
            park_name = park.split('_')[1]
            wind_col = park_name + '_Wind'
            solar_col = park_name + '_Solar'
            
            wind_gen = generation_data[wind_col].sum()
            solar_gen = generation_data[solar_col].sum()
            
            # 避免除以零的情况
            if wind_gen == 0:
                wind_gen = 1e-9  # 设为一个非常小的数以避免除以零
            if solar_gen == 0:
                solar_gen = 1e-9  # 设为一个非常小的数以避免除以零

            required_wind_capacity = max_load / (wind_gen * wind_cost_per_kw)
            required_solar_capacity = max_load / (solar_gen * solar_cost_per_kw)
            
            capacity[park_name] = {
                'wind': required_wind_capacity,
                'solar': required_solar_capacity
            }
    return capacity

def calculate_storage_capacity(load_data, storage_power_cost, storage_energy_cost, min_soc=0.1, max_soc=0.9):
    storage = {}
    for park in load_data.columns:
        if 'Load' in park:
            max_load = load_data[park].max()
            park_name = park.split('_')[1]
            # 假设储能容量应满足日最大负荷波动的50%
            storage_power = max_load * 0.8  # SOC变化范围乘以最大负荷
            storage_energy = storage_power * (max_soc + min_soc) / 2  # 假设SOC变化平均为(SOC上限+SOC下限)/2
            
            storage[park_name] = {
                'power': storage_power,
                'energy': storage_energy,
                'cost': storage_power * storage_power_cost + storage_energy * storage_energy_cost
            }
    return storage

# 定义单价
wind_cost_per_kw = 3000
solar_cost_per_kw = 2500
storage_power_cost = 800
storage_energy_cost = 1800

# 计算装机容量和储能系统容量
capacity = calculate_capacity(load_data_grown, generation_data, wind_cost_per_kw, solar_cost_per_kw)
print("Capacity:", capacity)

storage_capacity = calculate_storage_capacity(load_data_grown, storage_power_cost, storage_energy_cost)
print("Storage capacity:", storage_capacity)

# 经济性分析
def economic_analysis(capacity, storage_capacity, payback_period):
    total_cost = 0
    for park in capacity:
        wind_cost = capacity[park]['wind']
        solar_cost = capacity[park]['solar']
        storage_cost = storage_capacity[park]['cost']
        
        total_investment = wind_cost + solar_cost + storage_cost
        
        # 避免除以零的情况
        if payback_period == 0:
            payback_period = 1e-9  # 设为一个非常小的数以避免除以零

        annualized_cost = total_investment / payback_period
        
        total_cost += annualized_cost
    
    return total_cost

total_cost = economic_analysis(capacity, storage_capacity, 5)
print(f'总年化成本为：{total_cost}元')

# 计算联合园区的光伏装机容量和风电装机容量
total_wind_capacity = sum([capacity[park]['wind'] for park in capacity])
total_solar_capacity = sum([capacity[park]['solar'] for park in capacity])

print(f'联合园区的光伏装机容量为：{total_solar_capacity} kW')
print(f'联合园区的风电装机容量为：{total_wind_capacity} kW')



Load data columns: Index(['Load_A', 'Purchase_A', 'Different_A', 'Load_B', 'Purchase_B',
       'Different_B', 'Load_C', 'Purchase_C'],
      dtype='object')
Generation data columns: Index(['A_Solar', 'A_Wind', 'B_Solar', 'B_Wind', 'C_Solar', 'C_Wind'], dtype='object')
Capacity: {'A': {'wind': 223500000.0, 'solar': 0.05056465752908128}, 'B': {'wind': 0.033926026687394736, 'solar': 251399999.99999997}, 'C': {'wind': 0.043495452748121784, 'solar': 0.05526229567876515}}
Storage capacity: {'A': {'power': 536.4, 'energy': 268.2, 'cost': 911880.0}, 'B': {'power': 502.8, 'energy': 251.4, 'cost': 854760.0}, 'C': {'power': 607.2, 'energy': 303.6, 'cost': 1032240.0}}
总年化成本为：95539776.03664967元
联合园区的光伏装机容量为：251400000.1058269 kW
联合园区的风电装机容量为：223500000.0774215 kW


In [4]:
import pandas as pd

# 读取负荷数据
load_data = pd.read_excel('附件1：各园区典型日负荷数据.xlsx')
# 读取风光发电数据
generation_data = pd.read_excel('附件2_new.xlsx')

# 确保时间列不会影响计算
load_data.set_index('时间（h）', inplace=True)
generation_data.set_index('时间（h）', inplace=True)

# 打印列名以确认
print("Load data columns:", load_data.columns)
print("Generation data columns:", generation_data.columns)

# 负荷增长50%
load_data_grown = load_data * 1.5

# 计算每个园区需要的风电和光伏装机容量
def calculate_capacity(load_data, generation_data, wind_cost_per_kw, solar_cost_per_kw):
    capacity = {}
    for park in load_data.columns:
        if 'Load' in park:
            max_load = load_data[park].max()
            park_name = park.split('_')[1]
            wind_col = park_name + '_Wind'
            solar_col = park_name + '_Solar'
            
            wind_gen = generation_data[wind_col].sum()
            solar_gen = generation_data[solar_col].sum()
            
            # 避免除以零的情况
            if wind_gen == 0:
                wind_gen = 1e-9  # 设为一个非常小的数以避免除以零
            if solar_gen == 0:
                solar_gen = 1e-9  # 设为一个非常小的数以避免除以零

            required_wind_capacity = max_load / (wind_gen * wind_cost_per_kw)
            required_solar_capacity = max_load / (solar_gen * solar_cost_per_kw)
            
            capacity[park_name] = {
                'wind': required_wind_capacity,
                'solar': required_solar_capacity
            }
    return capacity

def calculate_storage_capacity(load_data, storage_power_cost, storage_energy_cost, min_soc=0.1, max_soc=0.9):
    storage = {}
    for park in load_data.columns:
        if 'Load' in park:
            max_load = load_data[park].max()
            park_name = park.split('_')[1]
            # 假设储能容量应满足日最大负荷波动的50%
            storage_power = max_load * 0.8  # SOC变化范围乘以最大负荷
            storage_energy = storage_power * (max_soc + min_soc) / 2  # 假设SOC变化平均为(SOC上限+SOC下限)/2
            
            storage[park_name] = {
                'power': storage_power,
                'energy': storage_energy,
                'cost': storage_power * storage_power_cost + storage_energy * storage_energy_cost
            }
    return storage

# 定义单价
wind_cost_per_kw = 3000
solar_cost_per_kw = 2500
storage_power_cost = 800
storage_energy_cost = 1800

# 计算装机容量和储能系统容量
capacity = calculate_capacity(load_data_grown, generation_data, wind_cost_per_kw, solar_cost_per_kw)
print("Capacity:", capacity)

storage_capacity = calculate_storage_capacity(load_data_grown, storage_power_cost, storage_energy_cost)
print("Storage capacity:", storage_capacity)

# 经济性分析
def economic_analysis(capacity, storage_capacity, payback_period, annual_load, annual_generation):
    total_cost = 0
    for park in capacity:
        wind_cost = capacity[park]['wind'] * wind_cost_per_kw
        solar_cost = capacity[park]['solar'] * solar_cost_per_kw
        storage_cost = storage_capacity[park]['cost']
        
        total_investment = wind_cost + solar_cost + storage_cost
        
        # 避免除以零的情况
        if payback_period == 0:
            payback_period = 1e-9  # 设为一个非常小的数以避免除以零

        annualized_cost = total_investment / payback_period
        
        total_cost += annualized_cost
    
    return total_cost

# 使用全年负荷和发电数据进行经济性分析
annual_load = load_data_grown.sum().sum()  # 年总负荷
annual_generation = generation_data.sum().sum()  # 年总发电量

total_cost = economic_analysis(capacity, storage_capacity, 5, annual_load, annual_generation)
print(f'总年化成本为：{total_cost}元')

# 计算联合园区的光伏装机容量和风电装机容量
total_wind_capacity = sum([capacity[park]['wind'] for park in capacity])
total_solar_capacity = sum([capacity[park]['solar'] for park in capacity])

print(f'联合园区的光伏装机容量为：{total_solar_capacity} kW')
print(f'联合园区的风电装机容量为：{total_wind_capacity} kW')


Load data columns: Index(['Load_A', 'Purchase_A', 'Different_A', 'Load_B', 'Purchase_B',
       'Different_B', 'Load_C', 'Purchase_C'],
      dtype='object')
Generation data columns: Index(['A_Solar', 'A_Wind', 'B_Solar', 'B_Wind', 'C_Solar', 'C_Wind'], dtype='object')
Capacity: {'A': {'wind': 223500000.0, 'solar': 0.05056465752908128}, 'B': {'wind': 0.033926026687394736, 'solar': 251399999.99999997}, 'C': {'wind': 0.043495452748121784, 'solar': 0.05526229567876515}}
Storage capacity: {'A': {'power': 536.4, 'energy': 268.2, 'cost': 911880.0}, 'B': {'power': 502.8, 'energy': 251.4, 'cost': 854760.0}, 'C': {'power': 607.2, 'energy': 303.6, 'cost': 1032240.0}}
总年化成本为：259800559875.36633元
联合园区的光伏装机容量为：251400000.1058269 kW
联合园区的风电装机容量为：223500000.0774215 kW


联合园区：

In [5]:
import pandas as pd

# 读取负荷数据
load_data = pd.read_excel('附件1：各园区典型日负荷数据.xlsx')
# 读取风光发电数据
generation_data = pd.read_excel('附件2_new.xlsx')

# 确保时间列不会影响计算
load_data.set_index('时间（h）', inplace=True)
generation_data.set_index('时间（h）', inplace=True)

# 打印列名以确认
print("Load data columns:", load_data.columns)
print("Generation data columns:", generation_data.columns)

# 负荷增长50%
load_data_grown = load_data * 1.5

# 计算购电量和弃风弃光电量
def calculate_purchase_and_abandonment(load_data, generation_data):
    purchase = {}
    abandonment = {}
    for park in load_data.columns:
        if 'Load' in park:
            park_name = park.split('_')[1]
            wind_col = park_name + '_Wind'
            solar_col = park_name + '_Solar'
            
            total_load = load_data[park].sum()
            total_wind_gen = generation_data[wind_col].sum()
            total_solar_gen = generation_data[solar_col].sum()
            total_gen = total_wind_gen + total_solar_gen
            
            purchase[park_name] = max(0, total_load - total_gen)
            abandonment[park_name] = max(0, total_gen - total_load)
            
    return purchase, abandonment

# 计算自发电成本
def calculate_self_generation_cost(generation_data, wind_cost_per_kwh=0.5, solar_cost_per_kwh=0.4):
    self_gen_cost = {}
    for col in generation_data.columns:
        if 'Wind' in col:
            park_name = col.split('_')[0]
            wind_gen = generation_data[col].sum()
            self_gen_cost[park_name + '_Wind'] = wind_gen * wind_cost_per_kwh
        elif 'Solar' in col:
            park_name = col.split('_')[0]
            solar_gen = generation_data[col].sum()
            self_gen_cost[park_name + '_Solar'] = solar_gen * solar_cost_per_kwh
            
    return self_gen_cost

# 计算购电成本
def calculate_purchase_cost(purchase, purchase_cost_per_kwh=1):
    purchase_cost = {}
    for park in purchase:
        purchase_cost[park] = purchase[park] * purchase_cost_per_kwh
    return purchase_cost

# 计算总供电成本
def calculate_total_supply_cost(self_gen_cost, purchase_cost):
    total_cost = 0
    for cost in self_gen_cost.values():
        total_cost += cost
    for cost in purchase_cost.values():
        total_cost += cost
    return total_cost

# 计算储能系统成本
def calculate_storage_cost(storage_capacity, storage_power_cost, storage_energy_cost):
    total_storage_cost = 0
    for park in storage_capacity:
        total_storage_cost += storage_capacity[park]['cost']
    return total_storage_cost

# 定义单价
wind_cost_per_kw = 3000
solar_cost_per_kw = 2500
storage_power_cost = 800
storage_energy_cost = 1800

# 计算装机容量和储能系统容量
capacity = calculate_capacity(load_data_grown, generation_data, wind_cost_per_kw, solar_cost_per_kw)
print("Capacity:", capacity)

storage_capacity = calculate_storage_capacity(load_data_grown, storage_power_cost, storage_energy_cost)
print("Storage capacity:", storage_capacity)

# 计算购电量和弃风弃光电量
purchase, abandonment = calculate_purchase_and_abandonment(load_data_grown, generation_data)
print("Purchase:", purchase)
print("Abandonment:", abandonment)

# 计算自发电成本
self_gen_cost = calculate_self_generation_cost(generation_data)
print("Self Generation Cost:", self_gen_cost)

# 计算购电成本
purchase_cost = calculate_purchase_cost(purchase)
print("Purchase Cost:", purchase_cost)

# 计算总供电成本
total_supply_cost = calculate_total_supply_cost(self_gen_cost, purchase_cost)
print(f"Total Supply Cost: {total_supply_cost}元")

# 计算储能系统成本
storage_cost = calculate_storage_cost(storage_capacity, storage_power_cost, storage_energy_cost)
print(f"Storage Cost: {storage_cost}元")

# 总成本（包含储能系统）
total_cost = total_supply_cost + storage_cost
print(f"总成本为：{total_cost}元")

# 经济性分析
def economic_analysis(total_cost, payback_period):
    annualized_cost = total_cost / payback_period
    return annualized_cost

annualized_cost = economic_analysis(total_cost, 5)
print(f"总年化成本为：{annualized_cost}元")


Load data columns: Index(['Load_A', 'Purchase_A', 'Different_A', 'Load_B', 'Purchase_B',
       'Different_B', 'Load_C', 'Purchase_C'],
      dtype='object')
Generation data columns: Index(['A_Solar', 'A_Wind', 'B_Solar', 'B_Wind', 'C_Solar', 'C_Wind'], dtype='object')
Capacity: {'A': {'wind': 223500000.0, 'solar': 0.05056465752908128}, 'B': {'wind': 0.033926026687394736, 'solar': 251399999.99999997}, 'C': {'wind': 0.043495452748121784, 'solar': 0.05526229567876515}}
Storage capacity: {'A': {'power': 536.4, 'energy': 268.2, 'cost': 911880.0}, 'B': {'power': 502.8, 'energy': 251.4, 'cost': 854760.0}, 'C': {'power': 607.2, 'energy': 303.6, 'cost': 1032240.0}}
Purchase: {'A': 11846.1959, 'B': 11558.8248, 'C': 11652.6895}
Abandonment: {'A': 0, 'B': 0, 'C': 0}
Self Generation Cost: {'A_Solar': 2.12164, 'A_Wind': 0.0, 'B_Solar': 0.0, 'B_Wind': 3.0876, 'C_Solar': 2.1975200000000004, 'C_Wind': 2.9083500000000004}
Purchase Cost: {'A': 11846.1959, 'B': 11558.8248, 'C': 11652.6895}
Total Supply C

In [8]:
import pandas as pd

# 读取负荷数据
load_data = pd.read_excel('附件1：各园区典型日负荷数据.xlsx')
# 读取风光发电数据
generation_data = pd.read_excel('附件2_new.xlsx')

# 确保时间列不会影响计算
load_data.set_index('时间（h）', inplace=True)
generation_data.set_index('时间（h）', inplace=True)

# 打印列名以确认
print("Load data columns:", load_data.columns)
print("Generation data columns:", generation_data.columns)

# 负荷增长50%
load_data_grown = load_data * 1.5

# 计算每个园区需要的风电和光伏装机容量
def calculate_capacity(load_data, generation_data, wind_cost_per_kw, solar_cost_per_kw):
    capacity = {}
    for park in load_data.columns:
        if 'Load' in park:
            max_load = load_data[park].max()
            park_name = park.split('_')[1]
            wind_col = park_name + '_Wind'
            solar_col = park_name + '_Solar'
            
            wind_gen = generation_data[wind_col].sum()
            solar_gen = generation_data[solar_col].sum()
            
            # 避免除以零的情况
            if wind_gen == 0:
                wind_gen = 1e-9  # 设为一个非常小的数以避免除以零
            if solar_gen == 0:
                solar_gen = 1e-9  # 设为一个非常小的数以避免除以零

            required_wind_capacity = max_load / (wind_gen * wind_cost_per_kw)
            required_solar_capacity = max_load / (solar_gen * solar_cost_per_kw)
            
            capacity[park_name] = {
                'wind': required_wind_capacity,
                'solar': required_solar_capacity
            }
    return capacity

# 计算储能系统容量
def calculate_storage_capacity(load_data, storage_power_cost, storage_energy_cost, min_soc=0.1, max_soc=0.9):
    storage = {}
    for park in load_data.columns:
        if 'Load' in park:
            max_load = load_data[park].max()
            park_name = park.split('_')[1]
            # 假设储能容量应满足日最大负荷波动的50%
            storage_power = max_load * 0.8  # SOC变化范围乘以最大负荷
            storage_energy = storage_power * (max_soc + min_soc) / 2  # 假设SOC变化平均为(SOC上限+SOC下限)/2
            
            storage[park_name] = {
                'power': storage_power,
                'energy': storage_energy,
                'cost': storage_power * storage_power_cost + storage_energy * storage_energy_cost
            }
    return storage

# 计算购电量和弃风弃光电量
def calculate_purchase_and_abandonment(load_data, generation_data):
    purchase = {}
    abandonment = {}
    for park in load_data.columns:
        if 'Load' in park:
            park_name = park.split('_')[1]
            wind_col = park_name + '_Wind'
            solar_col = park_name + '_Solar'
            
            total_load = load_data[park].sum()
            total_wind_gen = generation_data[wind_col].sum()
            total_solar_gen = generation_data[solar_col].sum()
            total_gen = total_wind_gen + total_solar_gen
            
            purchase[park_name] = max(0, total_load - total_gen)
            abandonment[park_name] = max(0, total_gen - total_load)
            
    return purchase, abandonment

# 计算自发电成本
def calculate_self_generation_cost(generation_data, wind_cost_per_kwh=0.5, solar_cost_per_kwh=0.4):
    self_gen_cost = {}
    for col in generation_data.columns:
        if 'Wind' in col:
            park_name = col.split('_')[0]
            wind_gen = generation_data[col].sum()
            self_gen_cost[park_name + '_Wind'] = wind_gen * wind_cost_per_kwh
        elif 'Solar' in col:
            park_name = col.split('_')[0]
            solar_gen = generation_data[col].sum()
            self_gen_cost[park_name + '_Solar'] = solar_gen * solar_cost_per_kwh
            
    return self_gen_cost

# 计算购电成本
def calculate_purchase_cost(purchase, purchase_cost_per_kwh=1):
    purchase_cost = {}
    for park in purchase:
        purchase_cost[park] = purchase[park] * purchase_cost_per_kwh
    return purchase_cost

# 计算总供电成本
def calculate_total_supply_cost(self_gen_cost, purchase_cost):
    total_cost = 0
    for cost in self_gen_cost.values():
        total_cost += cost
    for cost in purchase_cost.values():
        total_cost += cost
    return total_cost

# 计算储能系统成本
def calculate_storage_cost(storage_capacity, storage_power_cost, storage_energy_cost):
    total_storage_cost = 0
    for park in storage_capacity:
        total_storage_cost += storage_capacity[park]['cost']
    return total_storage_cost

# 定义单价
wind_cost_per_kw = 3000
solar_cost_per_kw = 2500
storage_power_cost = 800
storage_energy_cost = 1800

# 计算装机容量和储能系统容量
capacity = calculate_capacity(load_data_grown, generation_data, wind_cost_per_kw, solar_cost_per_kw)
print("Capacity:", capacity)

storage_capacity = calculate_storage_capacity(load_data_grown, storage_power_cost, storage_energy_cost)
print("Storage capacity:", storage_capacity)

# 计算购电量和弃风弃光电量
purchase, abandonment = calculate_purchase_and_abandonment(load_data_grown, generation_data)
print("Purchase:", purchase)
print("Abandonment:", abandonment)

# 计算自发电成本
self_gen_cost = calculate_self_generation_cost(generation_data)
print("Self Generation Cost:", self_gen_cost)

# 计算购电成本
purchase_cost = calculate_purchase_cost(purchase)
print("Purchase Cost:", purchase_cost)

# 计算总供电成本
total_supply_cost = calculate_total_supply_cost(self_gen_cost, purchase_cost)
print(f"Total Supply Cost: {total_supply_cost}元")

# 计算储能系统成本
storage_cost = calculate_storage_cost(storage_capacity, storage_power_cost, storage_energy_cost)
print(f"Storage Cost: {storage_cost}元")

# 总成本（包含储能系统）
total_cost = total_supply_cost + storage_cost
print(f"总成本为：{total_cost}元")

# 经济性分析
def economic_analysis(total_cost, payback_period):
    annualized_cost = total_cost / payback_period
    return annualized_cost

annualized_cost = economic_analysis(total_cost, 5)
print(f"总年化成本为：{annualized_cost}元")

# 计算联合园区的光伏装机容量和风电装机容量
total_wind_capacity = sum([capacity[park]['wind'] for park in capacity]) / 1000
total_solar_capacity = sum([capacity[park]['solar'] for park in capacity]) / 1000

print(f'联合园区的光伏装机容量为：{total_solar_capacity} KW')
print(f'联合园区的风电装机容量为：{total_wind_capacity} KW')


Load data columns: Index(['Load_A', 'Purchase_A', 'Different_A', 'Load_B', 'Purchase_B',
       'Different_B', 'Load_C', 'Purchase_C'],
      dtype='object')
Generation data columns: Index(['A_Solar', 'A_Wind', 'B_Solar', 'B_Wind', 'C_Solar', 'C_Wind'], dtype='object')
Capacity: {'A': {'wind': 223500000.0, 'solar': 0.05056465752908128}, 'B': {'wind': 0.033926026687394736, 'solar': 251399999.99999997}, 'C': {'wind': 0.043495452748121784, 'solar': 0.05526229567876515}}
Storage capacity: {'A': {'power': 536.4, 'energy': 268.2, 'cost': 911880.0}, 'B': {'power': 502.8, 'energy': 251.4, 'cost': 854760.0}, 'C': {'power': 607.2, 'energy': 303.6, 'cost': 1032240.0}}
Purchase: {'A': 11846.1959, 'B': 11558.8248, 'C': 11652.6895}
Abandonment: {'A': 0, 'B': 0, 'C': 0}
Self Generation Cost: {'A_Solar': 2.12164, 'A_Wind': 0.0, 'B_Solar': 0.0, 'B_Wind': 3.0876, 'C_Solar': 2.1975200000000004, 'C_Wind': 2.9083500000000004}
Purchase Cost: {'A': 11846.1959, 'B': 11558.8248, 'C': 11652.6895}
Total Supply C

In [1]:
import pandas as pd

# 读取负荷数据
load_data = pd.read_excel('附件1：各园区典型日负荷数据.xlsx')
# 读取风光发电数据
generation_data = pd.read_excel('附件2_new.xlsx')

# 确保时间列不会影响计算
load_data.set_index('时间（h）', inplace=True)
generation_data.set_index('时间（h）', inplace=True)

# 打印列名以确认
print("Load data columns:", load_data.columns)
print("Generation data columns:", generation_data.columns)

# 负荷增长50%
load_data_grown = load_data * 1.5

# 计算每个园区需要的风电和光伏装机容量
def calculate_capacity(load_data, generation_data, wind_cost_per_kw, solar_cost_per_kw):
    capacity = {}
    for park in load_data.columns:
        if 'Load' in park:
            max_load = load_data[park].max()
            park_name = park.split('_')[1]
            wind_col = park_name + '_Wind'
            solar_col = park_name + '_Solar'
            
            wind_gen = generation_data[wind_col].sum()
            solar_gen = generation_data[solar_col].sum()
            
            # 避免除以零的情况
            if wind_gen == 0:
                wind_gen = 1e-9  # 设为一个非常小的数以避免除以零
            if solar_gen == 0:
                solar_gen = 1e-9  # 设为一个非常小的数以避免除以零

            required_wind_capacity = max_load / (wind_gen * wind_cost_per_kw)
            required_solar_capacity = max_load / (solar_gen * solar_cost_per_kw)
            
            capacity[park_name] = {
                'wind': required_wind_capacity,
                'solar': required_solar_capacity
            }
    return capacity

def calculate_storage_capacity(load_data, storage_power_cost, storage_energy_cost, min_soc=0.1, max_soc=0.9):
    storage = {}
    optimal_storage_coefficient = {}
    for park in load_data.columns:
        if 'Load' in park:
            max_load = load_data[park].max()
            park_name = park.split('_')[1]
            # 假设储能容量应满足日最大负荷波动的50%
            storage_power = max_load * 0.8  # SOC变化范围乘以最大负荷
            storage_energy = storage_power * (max_soc + min_soc) / 2  # 假设SOC变化平均为(SOC上限+SOC下限)/2
            
            storage_cost = storage_power * storage_power_cost + storage_energy * storage_energy_cost
            storage[park_name] = {
                'power': storage_power,
                'energy': storage_energy,
                'cost': storage_cost
            }
            optimal_storage_coefficient[park_name] = storage_power / max_load  # 储能系数为储能功率占最大负荷的比例
    return storage, optimal_storage_coefficient

# 定义单价
wind_cost_per_kw = 3000
solar_cost_per_kw = 2500
storage_power_cost = 800
storage_energy_cost = 1800

# 计算装机容量和储能系统容量
capacity = calculate_capacity(load_data_grown, generation_data, wind_cost_per_kw, solar_cost_per_kw)
print("Capacity:", capacity)

storage_capacity, optimal_storage_coefficient = calculate_storage_capacity(load_data_grown, storage_power_cost, storage_energy_cost)
print("Storage capacity:", storage_capacity)
print("Optimal storage coefficient:", optimal_storage_coefficient)

# 经济性分析
def economic_analysis(capacity, storage_capacity, payback_period):
    total_cost = 0
    for park in capacity:
        wind_cost = capacity[park]['wind']
        solar_cost = capacity[park]['solar']
        storage_cost = storage_capacity[park]['cost']
        
        total_investment = wind_cost + solar_cost + storage_cost
        
        # 避免除以零的情况
        if payback_period == 0:
            payback_period = 1e-9  # 设为一个非常小的数以避免除以零

        annualized_cost = total_investment / payback_period
        
        total_cost += annualized_cost
    
    return total_cost

total_cost = economic_analysis(capacity, storage_capacity, 5)
print(f'总年化成本为：{total_cost}元')

# 计算联合园区的光伏装机容量和风电装机容量
total_wind_capacity = sum([capacity[park]['wind'] for park in capacity])
total_solar_capacity = sum([capacity[park]['solar'] for park in capacity])

print(f'联合园区的光伏装机容量为：{total_solar_capacity} kW')
print(f'联合园区的风电装机容量为：{total_wind_capacity} kW')


Load data columns: Index(['Load_A', 'Purchase_A', 'Different_A', 'Load_B', 'Purchase_B',
       'Different_B', 'Load_C', 'Purchase_C'],
      dtype='object')
Generation data columns: Index(['A_Solar', 'A_Wind', 'B_Solar', 'B_Wind', 'C_Solar', 'C_Wind'], dtype='object')
Capacity: {'A': {'wind': 223500000.0, 'solar': 0.05056465752908128}, 'B': {'wind': 0.033926026687394736, 'solar': 251399999.99999997}, 'C': {'wind': 0.043495452748121784, 'solar': 0.05526229567876515}}
Storage capacity: {'A': {'power': 536.4, 'energy': 268.2, 'cost': 911880.0}, 'B': {'power': 502.8, 'energy': 251.4, 'cost': 854760.0}, 'C': {'power': 607.2, 'energy': 303.6, 'cost': 1032240.0}}
Optimal storage coefficient: {'A': 0.7999999999999999, 'B': 0.8, 'C': 0.8}
总年化成本为：95539776.03664967元
联合园区的光伏装机容量为：251400000.1058269 kW
联合园区的风电装机容量为：223500000.0774215 kW


A,B,C三个园区单独的经济分析：

In [3]:
import pandas as pd

# 读取负荷数据
load_data = pd.read_excel('附件1：各园区典型日负荷数据.xlsx')
# 读取风光发电数据
generation_data = pd.read_excel('附件2_new.xlsx')

# 确保时间列不会影响计算
load_data.set_index('时间（h）', inplace=True)
generation_data.set_index('时间（h）', inplace=True)

# 打印列名以确认
print("Load data columns:", load_data.columns)
print("Generation data columns:", generation_data.columns)

# 负荷增长50%
load_data_grown = load_data * 1.5

# 计算购电量和弃风弃光电量
def calculate_purchase_and_abandonment(load_data, generation_data, park_name):
    purchase = 0
    abandonment = 0
    
    load_col = f"Load_{park_name}"
    wind_col = f"{park_name}_Wind"
    solar_col = f"{park_name}_Solar"
    
    total_load = load_data[load_col].sum()
    total_wind_gen = generation_data[wind_col].sum()
    total_solar_gen = generation_data[solar_col].sum()
    total_gen = total_wind_gen + total_solar_gen
    
    purchase = max(0, total_load - total_gen)
    abandonment = max(0, total_gen - total_load)
    
    return purchase, abandonment

# 计算自发电成本
def calculate_self_generation_cost(generation_data, park_name, wind_cost_per_kwh=0.5, solar_cost_per_kwh=0.4):
    self_gen_cost = 0
    
    wind_col = f"{park_name}_Wind"
    solar_col = f"{park_name}_Solar"
    
    wind_gen = generation_data[wind_col].sum()
    solar_gen = generation_data[solar_col].sum()
    
    self_gen_cost += wind_gen * wind_cost_per_kwh
    self_gen_cost += solar_gen * solar_cost_per_kwh
    
    return self_gen_cost

# 计算购电成本
def calculate_purchase_cost(purchase, purchase_cost_per_kwh=1):
    return purchase * purchase_cost_per_kwh

# 计算总供电成本
def calculate_total_supply_cost(self_gen_cost, purchase_cost):
    return self_gen_cost + purchase_cost

# 计算储能系统成本
def calculate_storage_cost(storage_capacity, storage_power_cost, storage_energy_cost):
    storage_cost = 0
    for park, data in storage_capacity.items():
        storage_cost += data['power'] * storage_power_cost
        storage_cost += data['energy'] * storage_energy_cost
    return storage_cost

# 计算装机容量
def calculate_capacity(load_data, generation_data):
    capacity = {}
    for park in load_data.columns:
        if 'Load' in park:
            park_name = park.split('_')[1]
            wind_col = f"{park_name}_Wind"
            solar_col = f"{park_name}_Solar"
            
            max_load = load_data[park].max()
            total_wind_gen = generation_data[wind_col].sum()
            total_solar_gen = generation_data[solar_col].sum()
            total_gen = total_wind_gen + total_solar_gen
            
            if total_gen == 0:
                wind_capacity = 0
                solar_capacity = 0
            else:
                avg_wind_gen = total_wind_gen / len(generation_data)
                avg_solar_gen = total_solar_gen / len(generation_data)
                
                wind_capacity = max_load / avg_wind_gen if avg_wind_gen != 0 else 0
                solar_capacity = max_load / avg_solar_gen if avg_solar_gen != 0 else 0
            
            capacity[park_name] = {
                'wind': wind_capacity,
                'solar': solar_capacity
            }
    return capacity

# 计算储能系统容量
def calculate_storage_capacity(load_data, storage_power_cost, storage_energy_cost, min_soc=0.1, max_soc=0.9):
    storage = {}
    for park in load_data.columns:
        if 'Load' in park:
            max_load = load_data[park].max()
            park_name = park.split('_')[1]
            storage_power = max_load * 0.8
            storage_energy = storage_power * (max_soc - min_soc)
            
            storage[park_name] = {
                'power': storage_power,
                'energy': storage_energy,
                'cost': storage_power * storage_power_cost + storage_energy * storage_energy_cost
            }
    return storage

# 定义单价
wind_cost_per_kw = 3000
solar_cost_per_kw = 2500
storage_power_cost = 800
storage_energy_cost = 1800

# 计算装机容量和储能系统容量
capacity = calculate_capacity(load_data_grown, generation_data)
print("Capacity:", capacity)

storage_capacity = calculate_storage_capacity(load_data_grown, storage_power_cost, storage_energy_cost)
print("Storage capacity:", storage_capacity)

# 经济性分析
def economic_analysis(total_cost, payback_period):
        if payback_period == 0:
            payback_period = 1e-9
        annualized_cost = total_cost / payback_period
        return annualized_cost

# 各园区经济状况分析
parks = ['A', 'B', 'C']
results = {}

for park in parks:
    print(f"Calculating for Park {park}...")
    
    purchase, abandonment = calculate_purchase_and_abandonment(load_data_grown, generation_data, park)
    print(f"Park {park} - Purchase: {purchase}, Abandonment: {abandonment}")
    
    self_gen_cost = calculate_self_generation_cost(generation_data, park)
    print(f"Park {park} - Self Generation Cost: {self_gen_cost}元")
    
    purchase_cost = calculate_purchase_cost(purchase)
    print(f"Park {park} - Purchase Cost: {purchase_cost}元")
    
    total_supply_cost = calculate_total_supply_cost(self_gen_cost, purchase_cost)
    print(f"Park {park} - Total Supply Cost: {total_supply_cost}元")
    
    storage_cost = calculate_storage_cost({park: storage_capacity[park]}, storage_power_cost, storage_energy_cost)
    print(f"Park {park} - Storage Cost: {storage_cost}元")
    
    total_cost = total_supply_cost + storage_cost
    print(f"Park {park} - 总成本为：{total_cost}元")
    
    annualized_cost = economic_analysis(total_cost, 5)
    print(f"Park {park} - 总年化成本为：{annualized_cost}元")
    
    results[park] = {
        'purchase': purchase,
        'abandonment': abandonment,
        'self_gen_cost': self_gen_cost,
        'purchase_cost': purchase_cost,
        'total_supply_cost': total_supply_cost,
        'storage_cost': storage_cost,
        'total_cost': total_cost,
        'annualized_cost': annualized_cost
    }

print("各园区经济状况分析结果:", results)

for park in parks:
    wind_capacity = capacity[park]['wind']
    solar_capacity = capacity[park]['solar']
    print(f'园区{park} 风电装机容量: {wind_capacity} kW')
    print(f'园区{park} 光伏装机容量: {solar_capacity} kW')



Load data columns: Index(['Load_A', 'Purchase_A', 'Different_A', 'Load_B', 'Purchase_B',
       'Different_B', 'Load_C', 'Purchase_C'],
      dtype='object')
Generation data columns: Index(['A_Solar', 'A_Wind', 'B_Solar', 'B_Wind', 'C_Solar', 'C_Wind'], dtype='object')
Capacity: {'A': {'wind': 0, 'solar': 3033.8794517448764}, 'B': {'wind': 2442.673921492421, 'solar': 0}, 'C': {'wind': 3131.6725978647682, 'solar': 3315.737740725909}}
Storage capacity: {'A': {'power': 536.4, 'energy': 429.12, 'cost': 1201536.0}, 'B': {'power': 502.8, 'energy': 402.24, 'cost': 1126272.0}, 'C': {'power': 607.2, 'energy': 485.76000000000005, 'cost': 1360128.0000000002}}
Calculating for Park A...
Park A - Purchase: 11846.1959, Abandonment: 0
Park A - Self Generation Cost: 2.12164元
Park A - Purchase Cost: 11846.1959元
Park A - Total Supply Cost: 11848.31754元
Park A - Storage Cost: 1201536.0元
Park A - 总成本为：1213384.31754元
Park A - 总年化成本为：242676.86350799998元
Calculating for Park B...
Park B - Purchase: 11558.8248